#datframe -> geodataframe

In [ ]:
!pip install geopandas

In [2]:
import pandas as pd
import geopandas as gpd

In [7]:
df = pd.read_csv("http://servizi.apss.tn.it/opendata/FARM001.csv")

In [9]:
df = df.drop(df[df["LATITUDINE_P"].str.contains("-")].index) #elimina righe del datframe con contenuto - nella colonna specificata

In [ ]:
df

In [11]:
from geopandas import GeoDataFrame
from shapely.geometry import Point

geometry = [Point(xy) for xy in zip(df["LONGITUDINE_P"], df["LATITUDINE_P"])]
df = df.drop(['LONGITUDINE_P', 'LATITUDINE_P'], axis=1)
farmacie = GeoDataFrame(df, crs="EPSG:4326", geometry=geometry) #epsg 4326: per coordinate geografiche

In [ ]:
farmacie

#esercizi

1. visualizzare su una mappa la posizione di una delle farmacie presenti nel dataset: l'utente inserisce parte del nome e il notebook visualizza una mappa della zona in cui si trova la farmacia con un marker nelal poszione della farmacia.

In [ ]:
farmacie

In [ ]:
!pip install contextily

In [18]:
import contextily

In [ ]:
nome = input("inserisci farmacia")
farmaciaScelta = farmacie.to_crs(3857)[farmacie.to_crs(3857)["FARMACIA"].str.contains(nome.upper())]
ax = farmaciaScelta.plot(figsize = (12, 12), color = "red", markersize = 10)
contextily.add_basemap(ax)

2. visualizzare tutte le farmacie presenti nel geodataset con i confini dei comuni in cui si trovano

In [22]:
comuni = gpd.read_file("/content/drive/MyDrive/Colab Notebooks/Com01012022_g")

In [ ]:
comuniFarmacie = comuni.to_crs(3857)[comuni.to_crs(3857).intersects(farmacie.to_crs(3857).unary_union)]
ax = comuniFarmacie.plot(figsize = (12, 12), edgecolor = "black", linewidth = 2, facecolor = "none")
farmacie.to_crs(3857).plot(ax = ax, markersize = 10, color = "red")
contextily.add_basemap(ax)

3. visualizzare un elenco dei comuni con il numero di farmacie presenti in ogni comune

In [59]:
farmacie = farmacie.drop("COMUNE", axis = 1)

In [ ]:
joined = gpd.sjoin(farmacie.to_crs(3857), comuniFarmacie, predicate = "intersects", how="left")
joined

In [74]:
farmacie_per_comune = joined.groupby("COMUNE")[["FARMACIA"]].count().sort_values(by = "FARMACIA", ascending = False).reset_index()
farmacie_per_comune

,COMUNE,FARMACIA
0,TRENTO,26
1,ROVERETO,11
2,ARCO,5
3,PERGINE VALSUGANA,5
4,RIVA DEL GARDA,4
...,...,...
96,COMANO TERME,1
97,CLES,1
98,CEMBRA LISIGNAGO,1
99,CAVEDAGO,1


4. visualizzare una mappa dei comuni con le superfici colorate in base al numero di farmacie che si trovano nel comune. Visualizzare anche la legenda.

In [56]:
comuniFarmacie["COMUNE"] = comuniFarmacie["COMUNE"].str.upper() #mette in maiuscolo tutti i comuni

In [ ]:
finale = comuniFarmacie.merge(farmacie_per_comune, on = "COMUNE")
finale

In [ ]:
ax = finale.plot(figsize = (12, 10), column = "FARMACIA", legend = True, alpha = 0.8, cmap = "Greens")
farmacie.to_crs(3857).plot(ax = ax, markersize = 10, color = "red")
contextily.add_basemap(ax)

5. visualizzare un grafico a torta con il numero di farmacie per ogni comune

In [70]:
import matplotlib.pyplot as plt

In [ ]:
dati = farmacie_per_comune["FARMACIA"]
stringhe = farmacie_per_comune["COMUNE"]
plt.figure(figsize=(16, 8))
plt.pie(dati, labels=stringhe, autopct='%1.1f%%')
plt.show()